<a href="https://colab.research.google.com/github/OlivierDataS/github_actions/blob/master/Projet_Sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation de l'environnement

In [2]:
# For Python
pip install pandas

# ORM for databases
pip install sqlalchemy # ORM for databases

#Python SQL Toolkit - SqlAlchemy installation
pip install ipython-sql

SyntaxError: ignored

Activation de SqlMagic

In [3]:
%load_ext sql
#%reload_ext sql
%config SqlMagic.displaycon = False
%config SqlMagic.feedback = False
%config SqlMagic.displaylimit = 50
%config SqlMagic.autocommit=True

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: UserWarning: Config option `displaycon` not recognized by `SqlMagic`.  Did you mean `displaylimit`?
  exec(code_obj, self.user_global_ns, self.user_ns)


Connection/creation à la BD/schema

In [4]:
%sql sqlite:///Itineraires.db

'Connected: @Itineraires.db'

Load tools for Python

In [5]:
import pandas as pd
from google.colab import files
import io
#from sqlalchemy import create_engine, ForeignKey
#from sqlalchemy import Column, Date, Integer, String
#from sqlalchemy.ext.declarative import declarative_base
import sqlite3
conn = sqlite3.connect('Itineraires.db')

In [ ]:
#engine = create_engine("sqlite:///Itineraires.db")
#connection = engine.raw_connection()

Création de la table de référentiel des villes

In [6]:
%%sql
CREATE TABLE IF NOT EXISTS city_repository (
city_id INTEGER PRIMARY KEY AUTOINCREMENT,
nom_commune VARCHAR,
code_postal INTEGER NOT NULL,
Lieu_dit VARCHAR,
gps_latitude FLOAT,
gps_longitude FLOAT,
Date_maj TIMESTAMP DEFAULT CURRENT_TIMESTAMP
);

 * sqlite:///Itineraires.db


[]

Import des données, soit par l'intermédiaire d'un script sql :

In [ ]:
%%sql
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST DENIS',97490,'BOIS DE NEFLES ST DENIS',-20.933029099,55.446977555);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST DENIS',97490,'ST DENIS CHAUDRON',-20.933029099,55.446977555);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST DENIS',97490,'LA BRETAGNE',-20.933029099,55.446977555);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST DENIS',97490,'MOUFIA',-20.933029099,55.446977555);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST DENIS',97490,'STE CLOTILDE',-20.933029099,55.446977555);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST JOSEPH',97480,'LES LIANES',-21.306243323,55.641977434);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST JOSEPH',97480,'VINCENDO',-21.306243323,55.641977434);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST JOSEPH',97480,,-21.306243323,55.641977434);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST JOSEPH',97480,'JEAN PETIT',-21.306243323,55.641977434);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST BENOIT',97470,,-21.091842698,55.649276452);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST BENOIT',97470,'BEAUFONDS',-21.091842698,55.649276452);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST PAUL',97460,,-21.044520611,55.322330771);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST PAUL',97460,'BELLEMENE',-21.044520611,55.322330771);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST PAUL',97460,'PLATEAU CAILLOUX',-21.044520611,55.322330771);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST LOUIS',97450,,-21.233608573,55.421584199);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST PHILIPPE',97442,,-21.303827331,55.745038691);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST PHILIPPE',97442,'BASSE VALLEE',-21.303827331,55.745038691);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('STE SUZANNE',97441,,-20.945801093,55.593086604);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST ANDRE',97440,,-20.963355482,55.642351104);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST ANDRE',97440,'LA CRESSONNIERE',-20.963355482,55.642351104);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST ANDRE',97440,'CAMBUSTON',-20.963355482,55.642351104);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('STE ROSE',97439,,-21.192230578,55.754535198);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('STE ROSE',97439,'LE PITON STE ROSE',-21.192230578,55.754535198);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('STE MARIE',97438,'RIVIERE DES PLUIES',-20.946751887,55.530655328);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('STE MARIE',97438,'LA GRANDE MONTEE',-20.946751887,55.530655328);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('STE MARIE',97438,,-20.946751887,55.530655328);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('STE MARIE',97438,'ROLAND GARROS AEROPORT',-20.946751887,55.530655328);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST BENOIT',97437,'STE ANNE',-21.091842698,55.649276452);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST LEU',97436,,-21.166060117,55.333293615);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST PAUL',97435,'TAN ROUGE',-21.044520611,55.322330771);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST PAUL',97435,'ST GILLES LES HAUTS',-21.044520611,55.322330771);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST PAUL',97435,'BERNICA',-21.044520611,55.322330771);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST PAUL',97434,'LA SALINE LES BAINS',-21.044520611,55.322330771);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST PAUL',97434,'ST GILLES LES BAINS',-21.044520611,55.322330771);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('SALAZIE',97433,'HELL BOURG',-21.046742072,55.5080505);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('SALAZIE',97433,,-21.046742072,55.5080505);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST PIERRE',97432,'RAVINE DES CABRIS',-21.312265663,55.49381116);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('LA PLAINE DES PALMISTES',97431,,-21.151967542,55.644258467);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('LE TAMPON',97430,'PONT D YVES',-21.223167157,55.558552873);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('LE TAMPON',97430,,-21.223167157,55.558552873);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('LE TAMPON',97430,'LES TROIS MARES',-21.223167157,55.558552873);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('LE TAMPON',97430,'TAMPON 14EME KM',-21.223167157,55.558552873);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('PETITE ILE',97429,,-21.339620557,55.568851128);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('PETITE ILE',97429,'PITON GOYAVES',-21.339620557,55.568851128);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('L ETANG SALE',97427,,-21.249009007,55.366858787);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('L ETANG SALE',97427,'L ETANG SALE LES BAINS',-21.249009007,55.366858787);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('LES TROIS BASSINS',97426,,-21.11011498,55.329677529);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('LES AVIRONS',97425,,-21.209718395,55.358631255);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('LES AVIRONS',97425,'TEVELAVE',-21.209718395,55.358631255);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST LEU',97424,'LE PITON ST LEU',-21.166060117,55.333293615
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST LEU',97424,'LE PLATE',-21.166060117,55.333293615);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST PAUL',97423,'LE GUILLAUME',-21.044520611,55.322330771);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST PAUL',97422,'LA SALINE',-21.044520611,55.322330771);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST LOUIS',97421,'LA RIVIERE',-21.233608573,55.421584199);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST LOUIS',97421,'LES MAKES',-21.233608573,55.421584199);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('LE PORT',97420,'LE PORT MARINE',-20.944125682,55.304499605);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('LE PORT',97420,'LE PORT ZUP',-20.944125682,55.304499605);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('LE PORT',97420,,-20.944125682,55.304499605);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('LA POSSESSION',97419,,-20.994208353,55.397683708);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('LA POSSESSION',97419,'LE DOS D ANE',-20.994208353,55.397683708);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('LA POSSESSION',97419,'STE THERESE',-20.994208353,55.397683708);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('LA POSSESSION',97419,'LA RIVIERE DES GALETS',-20.994208353,55.397683708);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('LE TAMPON',97418,'LA PLAINE DES CAFRES',-21.223167157,55.558552873);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('LE TAMPON',97418,'TAMPON 17EME KM',-21.223167157,55.558552873);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST DENIS',97417,'ST BERNARD',-20.933029099,55.446977555);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST DENIS',97417,'LA MONTAGNE',-20.933029099,55.446977555);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST LEU',97416,'LA CHALOUPE',-21.166060117,55.333293615);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ENTRE DEUX',97414,,-21.204038195,55.492359292);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('CILAOS',97413,'PALMISTE ROUGE',-21.143838284,55.458799844);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('CILAOS',97413,,-21.143838284,55.458799844);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('BRAS PANON',97412,,-21.023321429,55.619750752);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('BRAS PANON',97412,'RIVIERE DU MAT',-21.023321429,55.619750752);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST PAUL',97411,'BOIS DE NEFLES ST PAUL',-21.044520611,55.322330771);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST PIERRE',97410,,-21.312265663,55.49381116);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST PIERRE',97410,'BASSE TERRE',-21.312265663,55.49381116);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST PIERRE',97410,'GRAND BOIS',-21.312265663,55.49381116);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST PIERRE',97410,'TERRE SAINTE',-21.312265663,55.49381116);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST PIERRE',97410,'MONT VERT',-21.312265663,55.49381116);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST PIERRE',97410,'RAVINE BLANCHE',-21.312265663,55.49381116);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST DENIS',97400,,-20.933029099,55.446977555);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST DENIS',97400,'ST DENIS CAMELIAS',-20.933029099,55.446977555);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST DENIS',97400,'LE BRULE',-20.933029099,55.446977555);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST DENIS',97400,'BELLE PIERRE',-20.933029099,55.446977555);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST DENIS',97400,'ST FRANCOIS',-20.933029099,55.446977555);
INSERT INTO city_repository (nom_commune, code_postal, Lieu_dit, gps_latitude, gps_longitude) VALUES ('ST DENIS',97400,'ST DENIS TADAR',-20.933029099,55.446977555);

Import des données, soit par l'intermédiaire d'un fichier puis d'un dataframe (correspondance des nomps de colonnes) :

In [10]:
File_uploaded = files.upload()

Saving city_repository.csv to city_repository.csv


In [11]:
import_city_repository = pd.read_csv(io.BytesIO(File_uploaded['city_repository.csv']))

In [12]:
import_city_repository.to_sql('city_repository', conn, if_exists='append', index=False)

Vérifications

In [13]:
import_city_repository.head()

,nom_commune,code_postal,Lieu_dit,gps_latitude,gps_longitude
0,ST DENIS,97490,BOIS DE NEFLES ST DENIS,-20.933029,55.446978
1,ST DENIS,97490,ST DENIS CHAUDRON,-20.933029,55.446978
2,ST DENIS,97490,LA BRETAGNE,-20.933029,55.446978
3,ST DENIS,97490,MOUFIA,-20.933029,55.446978
4,ST DENIS,97490,STE CLOTILDE,-20.933029,55.446978


In [16]:
import_city_repository.count()

nom_commune      85
code_postal      85
Lieu_dit         61
gps_latitude     85
gps_longitude    85
dtype: int64

In [14]:
%%sql
SELECT * FROM city_repository
LIMIT 5
/* delete from city_repository */

 * sqlite:///Itineraires.db


city_id,nom_commune,code_postal,Lieu_dit,gps_latitude,gps_longitude,Date_maj
1,ST DENIS,97490,BOIS DE NEFLES ST DENIS,-20.933029099,55.446977555,2023-02-09 15:49:13
2,ST DENIS,97490,ST DENIS CHAUDRON,-20.933029099,55.446977555,2023-02-09 15:49:13
3,ST DENIS,97490,LA BRETAGNE,-20.933029099,55.446977555,2023-02-09 15:49:13
4,ST DENIS,97490,MOUFIA,-20.933029099,55.446977555,2023-02-09 15:49:13
5,ST DENIS,97490,STE CLOTILDE,-20.933029099,55.446977555,2023-02-09 15:49:13


In [15]:
%%sql
SELECT COUNT(*) FROM city_repository

 * sqlite:///Itineraires.db


COUNT(*)
85


===================================
Chargement des données POI - les tables sont crées depuis leurs dataframes.

Dataframe/table du fichier reunion_clean.csv

In [18]:
File_uploaded = files.upload()

Saving reunion_clean.csv to reunion_clean.csv


In [19]:
poi_reunion = pd.read_csv(io.BytesIO(File_uploaded['reunion_clean.csv']))

In [20]:
%sql persist poi_reunion

 * sqlite:///Itineraires.db


'Persisted poi_reunion'

Vérifications

In [21]:
poi_reunion.count()

ID                     1750
Nom_du_POI             1750
Latitude               1750
Longitude              1750
Adresse_postale        1655
Date_de_mise_a_jour    1750
Contacts_du_POI        1655
Description              29
region                 1750
commune                1750
code_postal            1750
category_0             1743
category_1             1224
category_2               78
dtype: int64

In [22]:
%%sql
SELECT count(*) FROM poi_reunion;

 * sqlite:///Itineraires.db


count(*)
1750


Dataframe/table du fichier reunion_activity.csv

In [23]:
File_uploaded = files.upload()

Saving reunion_activity.csv to reunion_activity.csv


In [24]:
poi_reunion_activity = pd.read_csv(io.BytesIO(File_uploaded['reunion_activity.csv']))

In [25]:
%sql persist poi_reunion_activity


 * sqlite:///Itineraires.db


'Persisted poi_reunion_activity'

Vérifications

In [26]:
poi_reunion_activity.count()

ID                     241
Nom_du_POI             241
Latitude               241
Longitude              241
Adresse_postale        233
Date_de_mise_a_jour    241
Contacts_du_POI        233
Description              1
region                 241
commune                241
code_postal            241
category_0             241
category_1             241
category_2               2
dtype: int64

In [27]:
%%sql
SELECT COUNT(*) FROM poi_reunion_activity

 * sqlite:///Itineraires.db


COUNT(*)
241


Dataframe/table du fichier reunion_coordonnees.csv

In [28]:
File_uploaded = files.upload()

Saving reunion_coordonnees.csv to reunion_coordonnees.csv


In [29]:
poi_reunion_coordonnees = pd.read_csv(io.BytesIO(File_uploaded['reunion_coordonnees.csv']))

In [30]:
%%sql
persist poi_reunion_coordonnees

 * sqlite:///Itineraires.db


'Persisted poi_reunion_coordonnees'

Vérifications

In [31]:
poi_reunion_coordonnees.count()

ID            1750
Nom_du_POI    1750
Latitude      1750
Longitude     1750
dtype: int64

In [32]:
%%sql
SELECT COUNT(*) FROM poi_reunion_coordonnees;

 * sqlite:///Itineraires.db


COUNT(*)
1750


Dataframe/table du fichier reunion_food.csv

In [33]:
File_uploaded = files.upload()

Saving reunion_food.csv to reunion_food.csv


In [34]:
poi_reunion_food = pd.read_csv(io.BytesIO(File_uploaded['reunion_food.csv']))

In [35]:
%%sql
persist poi_reunion_food

 * sqlite:///Itineraires.db


'Persisted poi_reunion_food'

Vérifications

In [36]:
poi_reunion_food.count()

ID                     1195
Nom_du_POI             1195
Latitude               1195
Longitude              1195
Adresse_postale        1195
Date_de_mise_a_jour    1195
Contacts_du_POI        1195
Description               0
region                 1195
commune                1195
code_postal            1195
category_0             1195
category_1              878
category_2               55
dtype: int64

In [37]:
%%sql
SELECT COUNT(*) FROM poi_reunion_food;

 * sqlite:///Itineraires.db


COUNT(*)
1195


Dataframe/table du fichier reunion_ville_cp.csv

In [38]:
File_uploaded = files.upload()

Saving reunion_ville_cp.csv to reunion_ville_cp.csv


In [39]:
poi_reunion_ville_cp = pd.read_csv(io.BytesIO(File_uploaded['reunion_ville_cp.csv']))

In [40]:
%%sql
persist poi_reunion_ville_cp

 * sqlite:///Itineraires.db


'Persisted poi_reunion_ville_cp'

Vérifications

In [41]:
poi_reunion_ville_cp.count()

code_postal    37
commune        37
region         37
dtype: int64

In [42]:
%%sql
SELECT COUNT(*) FROM poi_reunion_ville_cp

 * sqlite:///Itineraires.db


COUNT(*)
37
